In [1]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score

In [13]:
############### Load the dataset ###################
path_train = '1000_train.csv'
train_data=pd.read_csv(path_train)
samples = train_data.sample(n=700000)
print(samples.shape)

(700000, 28)


In [15]:
X =samples.iloc[:,1:]
Y =samples.iloc[:,0]
classes = 2
Y = to_categorical(Y,classes)

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(560000, 27) (560000, 2)
(140000, 27) (140000, 2)


In [17]:
s = MinMaxScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

In [18]:
dim = X_train.shape[1]
x = int(dim**(1/2)) +1
f = x*x
model = Sequential()
model.add(Dense(f-dim,name='feature', activation='relu',input_shape=(dim,)))
model.add(Dense(classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
# here, inputs and labels are same
model.fit(X_train,Y_train,epochs=5,batch_size=512,verbose=1)
extract = Model(model.inputs, model.get_layer('feature').output)
# predict whole inputs through it
x1 = extract.predict(X_train)
x2 = extract.predict(X_test)
# concatenate on horizontal axis
X1 = np.concatenate((X_train, x1), axis=1) 
X2 = np.concatenate((X_test, x2), axis=1) 
print(X1.shape,X2.shape)
train = X1.reshape(X1.shape[0],x,x)
test = X2.reshape(X2.shape[0],x,x)
print(train.shape,test.shape)

Epoch 1/5
1094/1094 [==============================] - 1s 1ms/step - loss: 0.3697 - accuracy: 0.8411
Epoch 2/5
1094/1094 [==============================] - 1s 1ms/step - loss: 0.2587 - accuracy: 0.8996
Epoch 3/5
1094/1094 [==============================] - 1s 1ms/step - loss: 0.2485 - accuracy: 0.9038
Epoch 4/5
1094/1094 [==============================] - 1s 1ms/step - loss: 0.2436 - accuracy: 0.9057
Epoch 5/5
1094/1094 [==============================] - 1s 1ms/step - loss: 0.2404 - accuracy: 0.9072
(560000, 36) (140000, 36)
(560000, 6, 6) (140000, 6, 6)


In [19]:
del X_train,X_test,X1,X2,x1,x2

In [20]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(train)):
    img = Image.fromarray(train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(test)):
    img = Image.fromarray(test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(560000, 32, 32, 3) (140000, 32, 32, 3)


In [21]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/20
2188/2188 [==============================] - 160s 73ms/step - loss: 0.2387 - accuracy: 0.9087 - val_loss: 0.2264 - val_accuracy: 0.9114
Epoch 2/20
2188/2188 [==============================] - 156s 71ms/step - loss: 0.2264 - accuracy: 0.9133 - val_loss: 0.2348 - val_accuracy: 0.9146
Epoch 3/20
2188/2188 [==============================] - 158s 72ms/step - loss: 0.2321 - accuracy: 0.9110 - val_loss: 0.2416 - val_accuracy: 0.9063
Epoch 4/20
2188/2188 [==============================] - 159s 73ms/step - loss: 0.2266 - accuracy: 0.9126 - val_loss: 0.2220 - val_accuracy: 0.9142
Epoch 5/20
2188/2188 [==============================] - 159s 73ms/step - loss: 0.2197 - accuracy: 0.9154 - val_loss: 0.2177 - val_accuracy: 0.9168
Epoch 6/20
2188/2188 [==============================] - 159s 73ms/step - loss: 0.2145 - accuracy: 0.9178 - val_loss: 0.2342 - val_accuracy: 0.9079
Epoch 7/20
2188/2188 [==============================] - 160s 73ms/step - loss: 0.2128 - accuracy: 0.9183 - val_loss: 0

In [22]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

17500/17500 [==============================] - 154s 9ms/step - loss: 0.1687 - accuracy: 0.9356
training accuracy:  0.9355928301811218
testing accuracy:  0.9200500249862671
